In [3]:
import os 
import glob 

import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rio
import netCDF4

from scipy.interpolate import interpn

import matplotlib.pyplot as plt
from   matplotlib.lines import Line2D

In [6]:
#### 
#  Open geogrid file
file     = '/home/sbarc/wrf/wrf401/sbareg/geo_em.d04.nc'
ds       = xr.open_dataset(file)
ds

#   read one 2DVar file to get coordinates
path     = '/home/sbarc/wrf/wrf401/sbareg/'
yr1      = 2007
m        = 0
files    = sorted(glob.glob(path + str(yr1+m) + '/wrfout_d04_2DVar_*.nc'))
f2       = xr.open_dataset(files[m])

#   creat xarray
terrain  = xr.DataArray(ds['HGT_M'][0,:,:].values, dims =("south_north","west_east"), name="terrain",     
                    coords={"XLAT":f2['XLAT'],"XLONG":f2['XLONG']},
                    attrs ={'terrain':'elevation (m)'})
#   creat landmask
landmask = xr.DataArray(ds['LANDMASK'][0,:,:].values, dims =("south_north","west_east"), name="landmask",     
                    coords={"XLAT":f2['XLAT'],"XLONG":f2['XLONG']},
                    attrs ={'landmask':'landmask (m)'})

In [17]:
T2 = f2['T2']
T2 = T2.sel(Time=12)
lats = T2['XLAT'].values[:,0]
lons = T2['XLONG'].values[0]

T2 = T2.assign_coords({'south_north':('south_north',lats)})
T2 = T2.assign_coords({'west_east':('west_east',lons)})
T2

<xarray.DataArray 'T2' (south_north: 234, west_east: 258)>
array([[287.81073, 287.80624, 287.79785, ..., 291.2735 , 291.28577, 291.28027],
       [287.81186, 287.80502, 287.7963 , ..., 291.3075 , 291.39163, 291.41388],
       [287.80984, 287.80292, 287.79504, ..., 291.30945, 291.41122, 291.4556 ],
       ...,
       [287.1055 , 287.14072, 287.18457, ..., 288.6221 , 288.39685, 288.53845],
       [287.12234, 287.15842, 287.18375, ..., 288.90588, 288.86835, 288.76636],
       [287.137  , 287.17136, 287.18005, ..., 290.1711 , 289.671  , 289.28918]],
      dtype=float32)
Coordinates:
    XLAT         (south_north, west_east) float32 33.51 33.51 ... 35.5 35.5
    XLONG        (south_north, west_east) float32 -121.3 -121.2 ... -118.6
  * south_north  (south_north) float32 33.51 33.52 33.53 ... 35.49 35.49 35.5
  * west_east    (west_east) float32 -121.3 -121.2 -121.2 ... -118.6 -118.6
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  TEMP at 2 M
    units:        K
    stagger:      
    coordinates:  XLONG XLAT XTIME

In [18]:
T2 = T2.rio.set_spatial_dims(x_dim='west_east', y_dim='south_north')
T2.rio.crs

In [19]:
T2.rio.write_crs('epsg:4326', inplace=True)

<xarray.DataArray 'T2' (south_north: 234, west_east: 258)>
array([[287.81073, 287.80624, 287.79785, ..., 291.2735 , 291.28577, 291.28027],
       [287.81186, 287.80502, 287.7963 , ..., 291.3075 , 291.39163, 291.41388],
       [287.80984, 287.80292, 287.79504, ..., 291.30945, 291.41122, 291.4556 ],
       ...,
       [287.1055 , 287.14072, 287.18457, ..., 288.6221 , 288.39685, 288.53845],
       [287.12234, 287.15842, 287.18375, ..., 288.90588, 288.86835, 288.76636],
       [287.137  , 287.17136, 287.18005, ..., 290.1711 , 289.671  , 289.28918]],
      dtype=float32)
Coordinates:
    XLAT         (south_north, west_east) float32 33.51 33.51 ... 35.5 35.5
    XLONG        (south_north, west_east) float32 -121.3 -121.2 ... -118.6
  * south_north  (south_north) float32 33.51 33.52 33.53 ... 35.49 35.49 35.5
  * west_east    (west_east) float32 -121.3 -121.2 -121.2 ... -118.6 -118.6
    spatial_ref  int64 0
Attributes:
    FieldType:     104
    MemoryOrder:   XY 
    description:   TEMP at 2 M
    units:         K
    stagger:       
    coordinates:   XLONG XLAT XTIME
    grid_mapping:  spatial_ref

In [20]:
outdir = '/home/patmccornack/data'
fname = '2007_T2_raster.tif'
T2.rio.to_raster(os.path.join(outdir, fname))